# Personal Social Services: Staff of Social Services Departments at 30 September - England, 2016

*Information on staff employed (directly and indirectly) by adult social services departments in England as at September 2016. This represents approximately 8 per cent of the overall Adult Social Care workforce.*

*CASSR - Council with Adult Social Services Responsibility.*

In [1]:
#Set this to find the info for your council
councilCrib='Wight'

In [2]:
#Original data file
#http://www.content.digital.nhs.uk/catalogue/PUB23291

In [3]:
import pandas as pd

#fn='http://www.content.digital.nhs.uk/catalogue/PUB23291/pss-staff-eng-16-tables.xlsx'

#For using a local copy...
#!mkdir -p data/
#!wget -P data/ http://www.content.digital.nhs.uk/catalogue/PUB23291/pss-staff-eng-16-tables.xlsx
fn='data/pss-staff-eng-16-tables.xlsx'

In [4]:
xl=pd.read_excel(fn,sheetname=None)
xl.keys()

dict_keys(['T2', 'T3a', 'T6c(ii)', 'Cover', 'T7a', 'T5', 'T4a', 'T7b(i) and (ii)', 'T6b(i)', 'T3b', 'T4b', 'T8b', 'T1', 'T9', 'T6b(ii)', 'T6a(i)', 'T8a', 'T6a(ii)', 'T6c(i)'])

In [5]:
cover=pd.read_excel(fn,sheetname='Cover').dropna(how='all').dropna(how='all',axis=1)
cover.iloc[14:35,0].tolist()

['Notes and definitions:',
 'Table 1: Total number of council-employed adult social services jobs, WTE jobs and people (Headcount)',
 'Table 2: Number of adult social services jobs by service group and job role group, CASSR level1',
 'Table 3a: Employees, starters and leavers, CASSR level',
 'Table 3b: Employees and Vacancies by Job Role, CASSR level',
 'Table 4a:  Total number of council-employed adult social services jobs, by employment status, England level',
 'Table 4b: Employment Status by Care Setting, England level',
 'Table 5: Capacity and Utilisation of Serivce, CASSR level',
 'Table 6a(i): Gender percentage distribution of adult social services jobs, by Job Role',
 'Table 6a(ii): Gender percentage distribution of adult social services jobs, by Job Role, CASSR level',
 'Table 6b(i): Age distribution of adult social services jobs, by Job Role',
 'Table 6b(ii): Age and Job Role, CASSR level',
 'Table 6c(i): Ethnicity of adult social services jobs, by Job Role',
 'Table 6c(ii): E

CASSR sheets have the local breakdown.

In [6]:
#Use full table name to generate sheet name and associate the two
cassrSheets=[(c,c.split(':')[0].replace('able ','')) for c in cover.iloc[:,0] if 'CASSR' in c]
cassrSheets

[('Table 2: Number of adult social services jobs by service group and job role group, CASSR level1',
  'T2'),
 ('Table 3a: Employees, starters and leavers, CASSR level', 'T3a'),
 ('Table 3b: Employees and Vacancies by Job Role, CASSR level', 'T3b'),
 ('Table 5: Capacity and Utilisation of Serivce, CASSR level', 'T5'),
 ('Table 6a(ii): Gender percentage distribution of adult social services jobs, by Job Role, CASSR level',
  'T6a(ii)'),
 ('Table 6b(ii): Age and Job Role, CASSR level', 'T6b(ii)'),
 ('Table 6c(ii): Ethnicity and Job Role, CASSR level', 'T6c(ii)'),
 ('Table 9: Qualifications of workers, by CASSR', 'T9')]

Define a function to get the actual data from a particular sheet.

In [7]:
def sheetCleaner(df):
    #The spreadsheet has the data midway down the sheet with multilevel column headings
    #Identify the lower column heading row
    cols=df[df.iloc[:,0]=='ONS Code']
    #Take all the data in the sheet down from the top multilevel heading
    df2=df.iloc[cols.index[0]-1:,:].reset_index(drop=True)
    #Set the top left column heading (currently blank)
    df2.iloc[0,0]='Base'
    #Fill across on top level column headings
    df2.iloc[0,:].fillna(method='ffill',inplace=True)
    #Set and clean the column headings - remove excessive whitespace and strip
    df2.columns=[c.str.replace(r'\W+',' ').str.strip() for c in [df2.iloc[0,:],df2.iloc[1,:]]]
    #The actual data is the datatable less the column headings...
    df2=df2[2:]
    #clean non-data rows at end
    df2.dropna(subset=[('Base','Region')],inplace=True)
    return df2

def df_grabber(fn,sheet):
    #Read in a particular sheet from the spreadsheet
    df=pd.read_excel(fn, sheetname=sheet,
                     na_values=['*','-']).dropna(how='all').dropna(axis=1,how='all').reset_index(drop=True)
    #Return a cleaned dataframe version of the data in that sheet
    return sheetCleaner(df)

In [8]:
df2=df_grabber(fn,'T3a')
df2[df2['Base','Council Name'].str.contains('Wight')]

0         Base                                                  \
1     ONS Code      Region Council Code           Council Name   
103  E06000046  South East          803  Isle of Wight Council   

0                                                               \
1   Total adult jobs 20164 Total starters 20154 Total leavers4   
103                    475                  155            160   

0   All Direct Care roles                  All Manager Supervisor roles  \
1    Sum of all employees Starters Leavers         Sum of all employees   
103                   300      100     100                           50   

0                    All Regulated professions                   \
1   Starters Leavers      Sum of all employees Starters Leavers   
103       10      15                        50       25      20   

0        All Other roles                   
1   Sum of all employees Starters Leavers  
103                   75       25      30

In [9]:
df2=df_grabber(fn,'T6c(ii)')
df2[df2['Base','Council Name'].str.contains('Wight')]

0         Base                                                  \
1     ONS Code      Region Council Code           Council Name   
103  E06000046  South East          803  Isle of Wight Council   

0          All Job Roles                                    \
1   Not recorded unknown White Mixed Multiple ethnic group   
103                  185   435                         NaN   

0                                                                    ...   \
1   Asian Asian British Black African Caribbean Black British Other  ...    
103                 NaN                                   NaN   NaN  ...    

0         Regulated Professions                      \
1   Mixed Multiple ethnic group Asian Asian British   
103                         NaN                 NaN   

0                                                              Other        \
1   Black African Caribbean Black British Other Not recorded unknown White   
103                                   NaN   NaN                   15    80   

0                                                    \
1   Mixed Multiple ethnic group Asian Asian British   
103                         NaN                 NaN   

0                                                
1   Black African Caribbean Black British Other  
103                                   NaN   NaN  

[1 rows x 34 columns]

In [10]:
admincols=['ONS Code', 'Region', 'Council Code', 'Council Name']

def subRowReport(group):
    nulls=[]
    for k in [c for c in group.index if c not in admincols]:
        if not pd.isnull(group.ix[k]):
            print('\t\t{}: {}'.format(k,group.ix[k]))
        else:
            nulls.append(k)
    if nulls: print('\t\tZero/unreported counts for: {}'.format(', '.join(nulls)))
    
def rowReport(row,sheetname):
    print('Report on {} ({}) for {} ({}, {})'.format(sheetname.split(':')[1].strip(),sheetname.split(':')[0].strip(),
                                                     row['Base','Council Name'],row['Base','ONS Code'],
                                                    row['Base','Region']))

The following report is a bare bones test to display cell headings and corresponding values for a particular value from each sheet, grouped by top-level heading.

We could use this as a basis for a [presentation style report](https://blog.ouseful.info/2017/01/20/a-recipe-for-automatically-going-from-data-to-text-to-reveal-js-slides/) containing automatically produced bullet points, charts etc.

In [11]:
from collections import OrderedDict

#Linearise the reports
for sheetname, sheet in cassrSheets:
    df2=df_grabber(fn,sheet)
    df2=df2[df2['Base','Council Name'].str.contains(councilCrib)]
    df2.apply(rowReport,sheetname=sheetname,axis=1)

    #Within each sheet, go through the top-level heading, and then do a subreport of cols in it
    #There's probably a pandas idiomatic way of doing this? Treat levels as groups?
    #For now, fudge it - create an ordered dict to represent: toplevelHeading:sublevelHeading
    #The ordering preserves the column order from the spreadsheet
    coldict=OrderedDict()
    for col in df2.columns:
        if col[0] not in coldict:
            coldict[col[0]]=[]
        coldict[col[0]].append(col[1])
    #Now we can iterate through the toplevel headings
    for k in coldict:
        #Ignoring the admin data stuff if there is only admin data
        if k=='Base':
            nonadmincols=[c for c in df2.xs('Base', level=0, axis=1).columns if c not in admincols]
            if not nonadmincols: continue
        print('\n\t{}'.format(k))
        df2.xs(k, level=0, axis=1).apply(subRowReport,axis=1)
        
    print('\n--------\n')
#'ONS Code','Region','Council Code','Council Name'

Report on Number of adult social services jobs by service group and job role group, CASSR level1 (Table 2) for Isle of Wight Council (E06000046, South East)

	All services
		All council job roles: 635
		Direct Care: 415
		Managers Supervisor: 60
		Regulated Professions: 70
		Other: 95

	Residential
		All job roles: 380
		Direct Care: 310
		Managers Supervisor: 25
		Other: 45
		Zero/unreported counts for: Regulated Professions

	Day
		All job roles: 10
		Direct Care: 5
		Zero/unreported counts for: Managers Supervisor, Regulated Professions, Other

	Domiciliary
		Zero/unreported counts for: All job roles, Direct Care, Managers Supervisor, Regulated Professions, Other

	Community
		All job roles: 225
		Direct Care: 90
		Managers Supervisor: 30
		Regulated Professions: 60
		Other: 45

	Other
		All job roles: 15
		Regulated Professions: 5
		Zero/unreported counts for: Direct Care, Managers Supervisor, Other

--------

Report on Employees, starters and leavers, CASSR level (Table 3a) for Is

## Report Magic

I wonder if we can do report magic? Eg by inspection of a table, write some canned prose around the column headings, then magic the actual values in? Note, null values would be reported in the templated text, which could get a bit clunky.

We'd also need to think about how to index keys into multiple tables...

In [12]:
tmp={'All council job roles':1,'Direct Care':2,'Managers Supervisor':3,'Regulated Professions':4,'Other':5}

ll='''
There were {All council job roles} job roles across the council, \
of which {Direct Care} were associated with direct care, \
{Managers Supervisor} were managerial/supervisory roles, \
{Regulated Professions} were in regulated professions and {Other} other.
'''.format(**tmp)
print(ll)


There were 1 job roles across the council, of which 2 were associated with direct care, 3 were managerial/supervisory roles, 4 were in regulated professions and 5 other.



In [13]:
#Can we do nested dicts?
tmp={'a':{'b a':1,'c':{'c c':3},'d':4},'e':0}
print('{a[b a]}, {a[c][c c]}'.format(**tmp))

#but the iniial key eg 'a' does not support spaces?

1, 3


This suggests we can generate a template to report on a simple or nested dict generated from the spreadsheet rows.

In [17]:
sheet='T6a(ii)'
ss=df_grabber(fn,sheet)
ss=ss[ss['Base','Council Name'].str.contains(councilCrib)]

print(ss.columns)
ss

MultiIndex(levels=[['All Job Roles', 'Base', 'Direct Care', 'Managers Supervisor', 'Other', 'Regulated Professions'], ['Council Code', 'Council Name', 'Female', 'Male', 'Not recorded Unknown', 'ONS Code', 'Region']],
           labels=[[1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 3, 3, 3, 5, 5, 5, 4, 4, 4], [5, 6, 0, 1, 3, 2, 4, 3, 2, 4, 3, 2, 4, 3, 2, 4, 3, 2, 4]],
           names=[0, 1])


0         Base                                                 All Job Roles  \
1     ONS Code      Region Council Code           Council Name          Male   
103  E06000046  South East          803  Isle of Wight Council           145   

0                               Direct Care                              \
1   Female Not recorded Unknown        Male Female Not recorded Unknown   
103    485                  NaN          80    335                  NaN   

0   Managers Supervisor                             Regulated Professions  \
1                  Male Female Not recorded Unknown                  Male   
103                  20     40                  NaN                    20   

0                               Other                              
1   Female Not recorded Unknown  Male Female Not recorded Unknown  
103     50                  NaN    30     65                  NaN

In [48]:
#Write the following template using paths to the appropriate table, top level headings and subheadings


l2='''
For *{r[T6a(ii)][name]}* ({r[T6a(ii)][key]}), across all job roles there were {r[T6a(ii)][All Job Roles][Male]} males and {r[T6a(ii)][All Job Roles][Female]} females \
({r[T6a(ii)][All Job Roles][Not recorded Unknown]} not recorded or unknown).

This breaks down as follows:

- in direct care, {r[T6a(ii)][Direct Care][Male]} males and {r[T6a(ii)][Direct Care][Female]} females \
({r[T6a(ii)][Direct Care][Not recorded Unknown]} not recorded or unknown)

- in managerial or supervisory roles, {r[T6a(ii)][Managers Supervisor][Male]} males and {r[T6a(ii)][Managers Supervisor][Female]} females \
({r[T6a(ii)][Managers Supervisor][Not recorded Unknown]} not recorded or unknown)

- amongst the regulated professions, {r[T6a(ii)][Regulated Professions][Male]} males and {r[T6a(ii)][Regulated Professions][Female]} females \
({r[T6a(ii)][Regulated Professions][Not recorded Unknown]} not recorded or unknown)

- and for other roles, {r[T6a(ii)][Other][Male]} males and {r[T6a(ii)][Other][Female]} females \
({r[T6a(ii)][Other][Not recorded Unknown]} not recorded or unknown)
'''

In [50]:
#This code churns a sheet into a dict that we can apply to the template.
cc={'r':{sheet:{'key':sheet, 'name':[k for k,v in cassrSheets if v==sheet][0]}}}

coldict=OrderedDict()
for col in ss.columns:
    if col[0] not in coldict:
        coldict[col[0]]=[]
    coldict[col[0]].append(col[1])

bb={}
#Now we can iterate through the toplevel headings
for k in coldict:
    #Ignoring the admin data stuff if there is only admin data
    if k=='Base':
        nonadmincols=[c for c in df2.xs('Base', level=0, axis=1).columns if c not in admincols]
        if not nonadmincols: continue
    bb[k]=ss[k].fillna(0).to_dict(orient='records')
    for v in bb:
        cc['r'][sheet][v]=bb[v][0]

        
#Now we can apply the dict obtained from the churned sheet to the template
print(l2.format(**cc))


For *Table 6a(ii): Gender percentage distribution of adult social services jobs, by Job Role, CASSR level* (T6a(ii)), across all job roles there were 145 males and 485 females (0 not recorded or unknown).

This breaks down as follows:

- in direct care, 80 males and 335 females (0 not recorded or unknown)

- in managerial or supervisory roles, 20 males and 40 females (0 not recorded or unknown)

- amongst the regulated professions, 20 males and 50 females (0 not recorded or unknown)

- and for other roles, 30 males and 65 females (0 not recorded or unknown)



This route suggests we can try to write boilerplate news release templates, with some text for each cell in a row for all sheets. However, for spreadsheets with large numbers of columns this can become unwieldy. Similarly, if a lot of cells contain null values, there will be a lot of redundant text in the report.

There is thus a tension between writing compact templated text and defining logic that determines when to apply those templates.

And once again, it seems like all we're doing is repeating things like [mail merge](https://support.office.com/en-us/article/Prepare-your-Excel-2016-data-source-for-a-Word-mail-merge-2d802b6b-a3a3-43e5-bb76-2cac7c68673e?ui=en-US&rs=en-US&ad=US), or starting down the path of trying to recreate templating frameworks like Django or Jinja2.

In [51]:
l2._meta.get_all_field_names()

AttributeError: 'str' object has no attribute '_meta'